In [2]:
import pygame
import random

pygame.init()

# --- Настройки экрана ---
WIDTH, HEIGHT = 600, 400
BLOCK_SIZE = 20
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game with Mouse Restart")

# --- Цвета ---
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (200, 0, 0)
BLUE = (0, 0, 255)

# --- Шрифты ---
font = pygame.font.SysFont("Arial", 24)
big_font = pygame.font.SysFont("Arial", 36)

# --- Таймер ---
clock = pygame.time.Clock()

# --- Генерация границ (стен по краям поля) ---
def generate_walls():
    walls = []
    for x in range(0, WIDTH, BLOCK_SIZE):
        walls.append((x, 0))  # верхняя
        walls.append((x, HEIGHT - BLOCK_SIZE))  # нижняя
    for y in range(0, HEIGHT, BLOCK_SIZE):
        walls.append((0, y))  # левая
        walls.append((WIDTH - BLOCK_SIZE, y))  # правая
    return walls

# --- Генерация еды вне стен и тела змеи ---
def generate_food(snake, walls):
    while True:
        x = random.randint(1, (WIDTH // BLOCK_SIZE) - 2) * BLOCK_SIZE
        y = random.randint(1, (HEIGHT // BLOCK_SIZE) - 2) * BLOCK_SIZE
        if (x, y) not in snake and (x, y) not in walls:
            return (x, y)

# --- Главная функция игры ---
def run_game():
    snake = [(100, 100)]  # стартовая позиция
    direction = (BLOCK_SIZE, 0)  # движение вправо
    walls = generate_walls()
    food = generate_food(snake, walls)
    score = 0
    level = 1
    speed = 10
    game_over = False

    while True:
        clock.tick(speed)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False  # выйти из всей игры

            # Перезапуск по клику мыши после проигрыша
            if event.type == pygame.MOUSEBUTTONDOWN and game_over:
                if event.button == 1:
                    return True

            # Управление стрелками
            if event.type == pygame.KEYDOWN and not game_over:
                if event.key == pygame.K_UP and direction != (0, BLOCK_SIZE):
                    direction = (0, -BLOCK_SIZE)
                elif event.key == pygame.K_DOWN and direction != (0, -BLOCK_SIZE):
                    direction = (0, BLOCK_SIZE)
                elif event.key == pygame.K_LEFT and direction != (BLOCK_SIZE, 0):
                    direction = (-BLOCK_SIZE, 0)
                elif event.key == pygame.K_RIGHT and direction != (-BLOCK_SIZE, 0):
                    direction = (BLOCK_SIZE, 0)

        if not game_over:
            # Движение змеи
            head = (snake[0][0] + direction[0], snake[0][1] + direction[1])
            snake.insert(0, head)

            # Проверка на проигрыш: врезка, выход за границу
            if (
                head in snake[1:] or
                head in walls or
                head[0] < 0 or head[0] >= WIDTH or
                head[1] < 0 or head[1] >= HEIGHT
            ):
                game_over = True
            elif head == food:
                score += 1
                if score % 4 == 0:
                    level += 1
                    speed += 2
                food = generate_food(snake, walls)
            else:
                snake.pop()

        # --- Отрисовка ---
        screen.fill(BLACK)

        # Стены
        for wall in walls:
            pygame.draw.rect(screen, BLUE, (*wall, BLOCK_SIZE, BLOCK_SIZE))

        # Змейка
        for part in snake:
            pygame.draw.rect(screen, GREEN, (*part, BLOCK_SIZE, BLOCK_SIZE))

        # Еда
        pygame.draw.rect(screen, RED, (*food, BLOCK_SIZE, BLOCK_SIZE))

        # Счёт и уровень
        score_text = font.render(f"Score: {score}  Level: {level}", True, WHITE)
        screen.blit(score_text, (10, 10))

        # Сообщение при проигрыше
        if game_over:
            msg = big_font.render("Game Over!", True, WHITE)
            restart_msg = font.render("Click to Restart or Close", True, WHITE)
            screen.blit(msg, (WIDTH // 2 - msg.get_width() // 2, HEIGHT // 2 - 40))
            screen.blit(restart_msg, (WIDTH // 2 - restart_msg.get_width() // 2, HEIGHT // 2 + 10))

        pygame.display.flip()

# --- Главный цикл: перезапуск при клике ---
while True:
    restart = run_game()
    if not restart:
        break

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
